In [1]:
from pytube import YouTube
import os

In [2]:
yt = YouTube('https://youtu.be/VFp38yj8h3A')

video = yt.streams.filter(only_audio=True).first()

out_file = video.download(output_path=".")

base, ext = os.path.splitext(out_file)
new_file = base + '.wav'
os.rename(out_file, new_file)

In [5]:

from vosk import Model, KaldiRecognizer
import json
import os

model = Model("eng")

# Large vocabulary free form recognition
rec = KaldiRecognizer(model, 16000)

# You can also specify the possible word list
#rec = KaldiRecognizer(model, 16000, "zero oh one two three four five six seven eight nine")

wf = open("test.wav", "rb")
wf.read(44) # skip header

while True:
    data = wf.read(4000)
    if len(data) == 0:
        break
    if rec.AcceptWaveform(data):
        res = json.loads(rec.Result())
        print (res['text'])

res = json.loads(rec.FinalResult())
print(res)

one zero zero zero one
nine oh two one oh
{'text': 'zero one eight zero three'}
